# Apply High-Frequency SNP Variants to Reference Genome (VCF → FASTA)

This notebook:
- Loads a multi-allelic VCF file with allele frequencies
- Loads a reference genome in FASTA format
- Applies the most common allele (based on frequency) at each SNP position
- Writes a new FASTA with updated sequences
- Logs which variants were applied

Tools Used: scikit-allel, Biopython


In [ ]:
import allel
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

##Load the VCF file

VCF = Variant Call Format  
read specific fields: chromosome, position, REF allele, ALT alleles, and allele frequency (AF).

In [ ]:
vcf_path = "path/to/g31016_tetraploids_multiallelic.vcf"

callset = allel.read_vcf(
    vcf_path,
    fields=['variants/CHROM', 'variants/POS', 'variants/REF', 'variants/ALT', 'variants/AF'],
    types={'variants/AF': 'f8'}
)

positions = callset['variants/POS']
ref_alleles = callset['variants/REF']
alt_alleles = callset['variants/ALT']
chroms = callset['variants/CHROM']
allele_freqs = callset['variants/AF']
variant_count = len(positions)

print("Loaded VCF with {} variants.".format(variant_count))


## Load the reference FASTA file

We store sequences as lists of characters so they can be modified in place.


In [ ]:
fasta_path = "path/to/g31016_tetraploids_biallelic.fasta"
fasta_dict = {}

with open(fasta_path, "r") as fasta_file:
    for rec in SeqIO.parse(fasta_file, "fasta"):
        fasta_dict[rec.id] = list(str(rec.seq))

print("Loaded {} contigs from FASTA.".format(len(fasta_dict)))


##Apply Variants Based on Allele Frequency

We'll only apply simple SNPs where:
- Top allele frequency > 5%
- It is at least 30% more common than the second allele
- It's a 1-to-1 SNP (not an indel or complex variant)


In [ ]:
modification_log = []
total_mods = 0

for idx in range(variant_count):
    pos = positions[idx]
    ref = ref_alleles[idx]
    raw_alt = alt_alleles[idx]
    chrom = chroms[idx]
    af = allele_freqs[idx]

    # Clean ALT alleles
    if isinstance(raw_alt, str):
        alt = raw_alt.strip().split(',') # coverts a string of nucleotides (eg "T,C,A") to  list of strings ["T", "C", "A"]
    else:
        alt = [str(a).strip() for a in raw_alt] # converts anything else to the format above 

    # Remove empty alleles
    alt = [a for a in alt if a] # get rid of empty strings 

    if not alt:
        continue  # If theres nothing left skip to the next variant 
    
    # Clean AF
    af_values = [af] if isinstance(af, float) else list(af) # is the allele frequency value a float, put it in a list. 
    allele_data = sorted(zip(alt, af_values), key=lambda x: x[1], reverse=True) #combine each ALT allele with is AF and sort them in reverse order (lambda x: x[1])

    top_alt, top_af = allele_data[0] #get the most frequent alternate allele and its frquency 
    second_af = allele_data[1][1] if len(allele_data) > 1 else 0.0 #If theres a second allele freq elese assign in 0.0

    # Filters
    if top_af <= 0.05 or (top_af - second_af) < 0.3: # AF must be > 0.05 and differnce between the two AF > 0.03
        continue
    if len(top_alt) != 1 or len(ref) != 1: # If the len of alt or ref alle not equal to 1, skip
        continue

    # Match contig
    contig_id = None
    offset = None
    for fid in fasta_dict:
        if fid.startswith(chrom): #Does the contig start with the same chromosome at this variant 
            try:
                region = fid.split(":")[1]
                start, end = map(int, region.split("-"))
                if start <= pos <= end:
                    contig_id = fid #does it match 
                    offset = pos - start
                    break
            except:
                continue

    # Apply change
    if contig_id and offset is not None and 0 <= offset < len(fasta_dict[contig_id]):
        fasta_dict[contig_id][offset] = top_alt
        modification_log.append((contig_id, pos, ref, top_alt))
        total_mods += 1


## Save Consensus FASTA and Log of Modifications

The output is:
- A new FASTA file with all accepted SNPs applied
- A text file listing all changes


In [ ]:
# Save new FASTA
output_fasta = "g31016_tetraploids_consensus_AFbased.fasta"
with open(output_fasta, "w") as out_fasta:
    for fid, chars in fasta_dict.items():
        record = SeqRecord(Seq("".join(chars)), id=fid, description="")
        SeqIO.write(record, out_fasta, "fasta")

# Save modification log
with open("modification_log_AFbased.txt", "w") as log_file:
    for contig, pos, ref, alt in modification_log:
        log_file.write("{}\t{}\t{} -> {}\n".format(contig, pos, ref, alt))

print("Done. {} variants applied.".format(total_mods))
